In [40]:
import ipywidgets as widgets
from IPython.display import display

import glob2
import pandas as pd

In [41]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100) #50) # default

In [80]:
def update_make_model(make_model):
    #print('create dataframe for {} csv files'.format(make_model))
    global filepath
    filepath = './output/*{}*.csv'.format(make_model)
    print(filepath)
    return None

make_models = ['Mazda-Miata', 'BMW-Z4', 'Nissan-370Z', 'Toyota-4Runner']
drop_down = \
widgets.Dropdown(
    options = make_models,
    value = make_models[0],
    description = 'Select Make-Model',
    disabled=False,
)


#display(drop_down)
widgets.interactive(update_make_model, make_model=drop_down)

interactive(children=(Dropdown(description='Select Make-Model', options=('Mazda-Miata', 'BMW-Z4', 'Nissan-370Z…

In [82]:
print(filepath)

./output/*Nissan-370Z*.csv


In [83]:
filenames = glob2.glob(filepath)
#print(filenames)

In [84]:
df = pd.read_csv(filenames[0])
for filename in filenames[1:]:
    df = pd.concat([df, pd.read_csv(filename)], ignore_index=True, sort=False)

In [85]:
df.index.name = 'orig_index'

In [86]:
#df.head(20)

In [87]:
agg = df.groupby('vin').agg({'date': ['max', 'min', 'count']})
agg.columns = ['max_date', 'min_date', 'count_date']

# add aggregations to row by vin
df_stats = df.join(agg, on='vin')


In [88]:
#df_stats.head(2)

In [89]:
def avail(row, date_max):
    if row['count_date'] == 1 and row['date'] == date_max:
        return 'new listing'
    if row['max_date'] < date_max:
        return 'listing not avail'
    return 'listing avail'

date_max = df['date'].max()
date_max

df_stats['availability'] = df_stats.apply(avail, axis=1, date_max=date_max)


In [90]:
#df_stats.head(2)

In [91]:
# order rows by vin and date to setup for calculating the price difference between dates
df_stats = df_stats.sort_values(by=['vin', 'date'])


In [92]:
#df_stats.head(15)

In [93]:
df_stats_price_diff = df_stats.groupby('vin')['vehicle_price'].rolling(2).apply(lambda price: price[1] - price[0], raw=True).to_frame('price_diff').reset_index()


In [94]:
#df_stats_price_diff.head(5)

In [95]:
assert len(df_stats) == len(df_stats_price_diff)

In [96]:
# join(..., left_on='index', right_on='level_1') is sufficient
# adding 'vin' is redundant, but keeps from adding duplicate columns vin_x, vin_y to dataframe
df_stats = df_stats.merge(df_stats_price_diff, left_on=['vin', 'orig_index'], right_on=['vin', 'orig_index'])


In [97]:
#df_stats.head(5)

In [98]:
#df_stats.columns

In [99]:
#['year_make_model_1', ''index', detail_url', 'vin', 'price_rating', 'vehicle_price', 'year_make_model_2', 'trim', 'location', 'mileage', 'exterior_color',
# 'interior_color', 'mpg', 'engine', 'transmission', 'drive_type', 'fuel_type', 'accident_check', 'usage', 'title', 'number_of_owners', 'date', 'max_date', 'min_date', 
# 'count_date', 'availability', 'price_diff']

# columns wanted for analysis
df_less_columns = df_stats[['year_make_model_1', 'vin', 'price_rating', 'vehicle_price', 'price_diff', 'trim', 'location', 'mileage', 
                 'exterior_color', 'interior_color', 'transmission', 'accident_check', 'usage', 'title', 'number_of_owners', 'date', 'availability', 'detail_url']]

In [100]:
#df_less_columns.head(5)

In [101]:
# various row filters
filter1 = df_less_columns['accident_check'] == 0 # no accidents
filter2 = df_less_columns['number_of_owners'] < 3 # less than three owners
filter3 = df_less_columns['title'] == 'Clean' # only interested in clean titles
filter4 = df_less_columns['availability'] != 'listing not avail' # not interested in listings that are no longer available


In [102]:
# note, for very large datasets it would be better to save new dataframe with the applied filter(s) instead of repeated filtering

#df_less_columns[filter1
#df_less_columns[filter1 & filter2]
#df_less_columns[filter1 & filter2 & filter3]
vins = list(set(df_less_columns[filter1 & filter2 & filter3 & filter4]['vin']))
list(enumerate(vins))

[(0, 'JN1AZ4FH9CM371012'),
 (1, 'JN1AZ4FH4AM301379'),
 (2, 'JN1AZ4FH5EM621137'),
 (3, 'JN1AZ4FH5EM621073'),
 (4, 'JN1AZ4FH2BM360030'),
 (5, 'JN1AZ4FH0BM360219')]

In [117]:

def update_vin(vin):
    #print('create dataframe for {} csv files'.format(make_model))
    global sel_vin
    sel_vin = vin
    print(sel_vin)
    return None

vins = list(set(df_less_columns[filter1 & filter2 & filter3 & filter4]['vin']))
drop_down = \
widgets.Dropdown(
    options = vins,
    value = vins[0],
    description = 'Select vins',
    disabled=False,
)


#display(drop_down)
widgets.interactive(update_vin, vin=drop_down)

interactive(children=(Dropdown(description='Select vins', options=('JN1AZ4FH9CM371012', 'JN1AZ4FH4AM301379', '…

In [115]:
#filter5 = df_less_columns['vin'] == vins[1]
filter5 = df_less_columns['vin'] == sel_vin
df_less_columns[filter5] #['detail_url']

,year_make_model_1,vin,price_rating,vehicle_price,price_diff,trim,location,mileage,exterior_color,interior_color,transmission,accident_check,usage,title,number_of_owners,date,availability,detail_url
367,2012 Nissan 370Z,JN1AZ4FH9CM371012,Excellent Price,17999,NaN,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-21 09:58:55.542922,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
368,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17999,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-23 18:44:30.671126,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
369,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17999,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-25 08:17:56.753564,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
370,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17999,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-29 20:02:29.517854,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
371,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17999,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-30 07:11:23.209785,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
372,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17899,-100.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-03-31 09:49:22.664277,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
373,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17899,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-04-02 16:07:42.515195,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
374,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17899,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-04-07 13:17:51.853408,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
375,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17899,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-04-10 10:31:44.778619,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/
376,2012 Nissan 370Z,JN1AZ4FH9CM371012,Great Price,17899,0.0,Touring Roadster Auto,"Somerset, MA",46783,Black Cherry Metallic,Wine,Automatic,0,Personal or Rental Use,Clean,1,2020-04-14 08:15:04.305218,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JN1AZ4FH9CM371012/2012-nissan-370z/


In [104]:
# filter by index range
#df_less_columns[filter1 & filter2 & filter3 & filter4].loc[66:79] # view range of listings of interest

In [105]:
# filter by index row
#df_less_columns[filter1 & filter2 & filter3 & filter4].loc[109] # view listings details for a specific listing